<a href="https://colab.research.google.com/github/TwistedGrims/sdfsdf32423dsfsdzxvczxfweqr2314/blob/main/imagegenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import Google drive
from google.colab import drive
drive.mount('/content/drive')
!pip install -U xformers



In [ ]:
#@title Load Model
import torch
import random
from diffusers import StableDiffusionXLPipeline
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

safetensor_path = "/content/drive/MyDrive/waiNSFWIllustrious_v150.safetensors"

pipe = StableDiffusionXLPipeline.from_single_file(
    safetensor_path,
    torch_dtype=torch_dtype
).to(device)
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

pipe.enable_xformers_memory_efficient_attention()
pipe.enable_vae_slicing()
pipe.enable_sequential_cpu_offload()



In [ ]:
# =========================
# Generation Settings
# =========================
prompt = "lazypos, Hatsune Miku, neon, glowing, futuristic city"  #@param {type:"string"}
negative_prompt = "lazyneg, blurry, bad anatomy, low quality"  #@param {type:"string"}

aspect_ratios = {
    "1024x1024": (1024, 1024),
    "1216x832": (1216, 832),
    "832x1216": (832, 1216)
}
chosen_ratio = "1216x832"  #@param ["1024x1024", "1216x832", "832x1216"]
width, height = aspect_ratios[chosen_ratio]

num_rows = 1  #@param {type:"integer"}
num_cols = 1  #@param {type:"integer"}
cfg_scale = 7  #@param {type:"number"}
steps = 30  #@param {type:"integer"}
seed = 0  #@param {type:"integer"}

# =========================
# Generate Images
# =========================
all_images = []
for _ in range(num_rows):
    row_images = []
    for _ in range(num_cols):
        if seed and seed > 0:
            current_seed = seed
        else:
            current_seed = random.randint(0, 2**32 - 1)

        generator = torch.Generator(device=device).manual_seed(current_seed)

        result = pipe(
            prompt=[prompt],
            negative_prompt=[negative_prompt],
            width=width,
            height=height,
            num_inference_steps=steps,
            guidance_scale=cfg_scale,
            generator=generator
        )

        img = result.images[0]
        row_images.append(img)

    all_images.extend(row_images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid
